In [ ]:
# load 3 individual datasets needed to construct a dataset for model training

import pandas as pd

power = pd.read_csv('/content/drive/MyDrive/하늘/data files/power_generation.csv')
climate = pd.read_csv('/content/drive/MyDrive/하늘/data files/climate.csv')
sunrise_sunset = pd.read_csv('/content/drive/MyDrive/하늘/data files/sunrise_sunset.csv')

In [ ]:
climate['date'] = pd.to_datetime(climate['date'])
power['date'] = pd.to_datetime(power['date'])

In [ ]:
sunrise_sunset['date'] = pd.to_datetime(sunrise_sunset['date'], format='%Y%m%d')
sunrise_sunset['sunrise'] = pd.to_datetime(sunrise_sunset['date'].astype(str) + ' ' + sunrise_sunset['sunrise'], format='%Y-%m-%d %H:%M')
sunrise_sunset['sunset'] = pd.to_datetime(sunrise_sunset['date'].astype(str) + ' ' + sunrise_sunset['sunset'], format='%Y-%m-%d %H:%M')

In [ ]:
# merge power & sunrise_sunset

power['date_only'] = power['date'].dt.date
power['date_only'] = pd.to_datetime(power['date_only'])

merged_df = power.merge(sunrise_sunset, left_on='date_only', right_on='date', how='inner')

merged_df = merged_df.drop('date_y', axis=1)

In [ ]:
# extract only dataset within the sunrise-sunset time

filtered_df = merged_df[(merged_df['date_x'] >= merged_df['sunrise']) & (merged_df['date_x'] <= merged_df['sunset'])]
filtered_df = filtered_df.drop(filtered_df.columns[0], axis=1)
filtered_df = filtered_df.rename(columns={filtered_df.columns[0]: 'date'})

In [ ]:
# delete rows with all values for generation are 0

filtered_df = filtered_df.groupby(['date_only', '발전기']).filter(lambda x: x['발전량(MWh)'].ne(0).any())
print(filtered_df)

                      date          발전기   발전량(MWh)      총량(MW)   평균(MW)  \
80     2022-01-31 08:00:00     삼천포태양광 3      7.380    1546.092     64.0   
81     2022-01-31 09:00:00     삼천포태양광 3    123.228    1546.092     64.0   
82     2022-01-31 10:00:00     삼천포태양광 3    215.136    1546.092     64.0   
83     2022-01-31 11:00:00     삼천포태양광 3    254.124    1546.092     64.0   
84     2022-01-31 12:00:00     삼천포태양광 3    233.748    1546.092     64.0   
...                    ...          ...        ...         ...      ...   
326870 2023-10-01 14:00:00  고흥만 수상태양광 1  43067.472  357075.348  14878.0   
326871 2023-10-01 15:00:00  고흥만 수상태양광 1  33452.412  357075.348  14878.0   
326872 2023-10-01 16:00:00  고흥만 수상태양광 1  20066.676  357075.348  14878.0   
326873 2023-10-01 17:00:00  고흥만 수상태양광 1   5898.564  357075.348  14878.0   
326874 2023-10-01 18:00:00  고흥만 수상태양광 1     51.156  357075.348  14878.0   

         최대(시간별)  최소(시간별)         최대   최소  date_only             sunrise  \
80        254.12    7.3

In [ ]:
# create a dictionary of generator:region

generator_to_region = {
    '삼천포': 'Namhae',
    '영흥': 'Incheon',
    '구미': 'Gumi',
    '두산': 'Changwon',
    '광양항': 'Gwangyang',
    '영동': 'Youngdong',
    '경상대': 'Jinju',
    '예천': 'Yecheon',
    '고흥': 'Goheung'
}

In [ ]:
# make a region column corresponding to the generator

def get_region(generator_name):
    for keyword, region in generator_to_region.items():
        if keyword in generator_name:
            return region
    return None

filtered_df['region'] = filtered_df['발전기'].apply(get_region)

print(filtered_df[df_cols])

                      date          발전기   region
80     2022-01-31 08:00:00     삼천포태양광 3   Namhae
81     2022-01-31 09:00:00     삼천포태양광 3   Namhae
82     2022-01-31 10:00:00     삼천포태양광 3   Namhae
83     2022-01-31 11:00:00     삼천포태양광 3   Namhae
84     2022-01-31 12:00:00     삼천포태양광 3   Namhae
...                    ...          ...      ...
326870 2023-10-01 14:00:00  고흥만 수상태양광 1  Goheung
326871 2023-10-01 15:00:00  고흥만 수상태양광 1  Goheung
326872 2023-10-01 16:00:00  고흥만 수상태양광 1  Goheung
326873 2023-10-01 17:00:00  고흥만 수상태양광 1  Goheung
326874 2023-10-01 18:00:00  고흥만 수상태양광 1  Goheung

[144971 rows x 3 columns]


In [ ]:
# merging climate data to corresponding region & date of power generation data

climate_cols = ['temperature', 'precipitation', 'wind speed',
       'humidity', 'air pressure', 'sunshine', 'insolation', 'snowfall',
       'cloud', 'ground temperature']

for index, row in filtered_df.iterrows():
  # print(f"current row : {index}")
  date = row['date']
  region = row['region']

  climate_row = climate[(climate['date'] == date) & (climate['region'] == region)]
  # print("  matching row found")

  for col in climate_cols:
    # print(f"    current col : {col}")
    if not climate_row.empty:
      filtered_df.at[index, col] = climate_row[col].values[0]
    else:
      filtered_df.at[index, col] = None

final_df = filtered_df

In [ ]:
# drop 여수, 탑선

condition = (final_df['발전기'] == '여수태양광 1') | (final_df['발전기'] == '탑선태양광 1') | (final_df['발전기'] == '탑선태양광 3')
final_df = final_df[~condition]

In [ ]:
final_df.to_csv("merged_dataset.csv")